# Codes for bootcamp talk: Advanced LLM & Agent Systems Bootcamp
By: Lior Gazit.  
Repo: [agentic_actions_locally_hosted](https://github.com/LiorGazit/agentic_actions_locally_hosted)  

<a target="_blank" href="https://colab.research.google.com/github/LiorGazit/agentic_actions_locally_hosted/blob/main/agents_building_workshop/Codes_for_the_Bootcamp_talk.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a> (pick a GPU Colab session for fastest computing)  

```
Disclaimer: The content and ideas presented in this notebook are solely those of the author, Lior Gazit, and do not represent the views or intellectual property of the author's employer.
``` 

## II. Hands-On Python Demonstrations
### Demo 1: RAG Pipeline with Chained Prompt Processing

Installing:

In [1]:
!pip -q install sentence-transformers faiss-cpu langchain tiktoken -U "autogen-agentchat" "autogen-ext[openai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

If this notebook is run outside of the repo's codes, get the necessary code from the remote repo:

In [2]:
import os
import requests

# If the module isn't already present (e.g. in Colab), fetch it from GitHub
if not os.path.exists("spin_up_LLM.py"):
    url = "https://raw.githubusercontent.com/LiorGazit/agentic_actions_locally_hosted/refs/heads/main/spin_up_LLM.py"
    resp = requests.get(url)
    resp.raise_for_status()
    with open("spin_up_LLM.py", "w") as f:
        f.write(resp.text)
    print("Downloaded spin_up_LLM.py from GitHub")

Downloaded spin_up_LLM.py from GitHub


Setting up the Vector Store and RAG pipeline:

In [3]:
# Import required libraries
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Example documents (could be clinical notes, financial filings, etc.)
documents = [
    "Patient has diabetes type 2 and shows high glucose levels.",
    "Recent financial filings show revenue growth despite supply chain issues.",
    "Patient diagnosed with hypertension, recommended lifestyle changes.",
    "The company's earnings call mentioned concerns over increased production costs.",
]

# Step 1: Create embeddings using SentenceTransformer
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # lightweight embedding model

# Generate embeddings for the documents
document_embeddings = embedding_model.encode(documents)

# Step 2: Setup FAISS vector store
dimension = document_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(document_embeddings)

# Step 3: Define the retriever(query) function
def retriever(query, top_k=2):
    # Generate embedding for the query
    query_embedding = embedding_model.encode([query])

    # Perform the similarity search in the FAISS index
    distances, indices = faiss_index.search(query_embedding, top_k)

    # Retrieve the top_k most similar documents
    retrieved_docs = [documents[idx] for idx in indices[0]]

    return retrieved_docs

# Example usage of the retriever
query = "What did the company say about production costs?"
context_docs = retriever(query)
print("\n\nRetrieved documents for context:\n", context_docs)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]



Retrieved documents for context:
 ["The company's earnings call mentioned concerns over increased production costs.", 'Recent financial filings show revenue growth despite supply chain issues.']


Setting up the Vector Store and RAG pipeline:

In [4]:
from spin_up_LLM import spin_up_LLM
from langchain_core.prompts import ChatPromptTemplate

query = "What is the patient's diagnosis given these notes?"
context_docs = retriever(query)
question = f"Using the following context, answer the question:\n\n{context_docs}\n\nQ: {query}\n\n---\nA:"
local_llm = spin_up_LLM(chosen_llm="gemma3")

answer_local = local_llm.generate([question])
print(answer_local.generations[0][0].text)

🚀 Installing Ollama...
🚀 Starting Ollama server...
→ Ollama PID: 1335
⏳ Waiting for Ollama to be ready…
🚀 Pulling model 'gemma3'…
Available models:
NAME             ID              SIZE      MODIFIED               
gemma3:latest    a2af6cc3eb7f    3.3 GB    Less than a second ago    

🚀 Installing langchain-ollama…
A: The patient has diabetes type 2 and hypertension.


Prompting using OpenAI's API (paid) route:

In [5]:
# In Colab, use getpass to securely prompt for your API key
from getpass import getpass
import openai

openai.api_key = getpass("Paste your OpenAI API key: ")

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role":"system","content":"You are a medical assistant."},
        {"role":"user",  "content": question}
    ]
)

answer_api = response.choices[0].message.content
print(answer_api)

Paste your OpenAI API key: ··········
The patient's diagnosis is type 2 diabetes and hypertension.


## Demo 2: Multi-Agent Team Interaction (Agent Collaboration)